# Konkurs | 6 edycja kursu "Praktyczne uczenie maszynowe"


### Cel:
Prognozowanie wartości zwrotu dla danego zamówienia. 


## Na czym polega konkurs?
Jest sklep online, który sprzedaje różne produkty dla ludzi z różnych krajów. Jedno zamówienie (faktura) może zawierać więcej niż jeden produkt. Trend, który jest dość popularny w szczególności w krajach zachodnich, to możliwość zwrotów i klienci chętnie z tego korzystają. Zwroty generują koszty i warto umieć przewidzieć to i jakoś z tym podziałać (pomijając skrajne przypadki takie jak "nie sprzedawać" klientowi X, bo to pewnie jest zakazane prawem). Natomiast pewnie dałoby się coś zrobić, jeśli wiedzielibyśmy, które zamówienia mogą być z dużym prawdopodobieństwem zwrócone.

**To zadanie można podzielić na 2 części**: z jednej strony jest to klasyfikacja, ponieważ ostatecznie trzeba stwierdzić, **czy ten produkt będzie zwrócony czy nie**. Z drugiej strony (i to jest docelowe zadanie) przez to, że zamówienie może zawierać więcej niż jeden produkt, to **ostatecznie chcemy obliczyć sumaryczny koszt zwrotów**. Możesz założyć, że jeśli w danym złożonym i opłaconym zamówieniu jest zwrot nawet jednego produktu, to zwracana jest całość zamówienia (czyli wszystkie produkty zawarte w tym zamówieniu). **Suma cen tych produktów daje całkowitą wartość zwrotu, właśnie to będziemy przewidydwać**.



### Metryką sukcesu
będzie **mean squared error**.

Dlaczego taka metryka? 
- Po pierwsze którąś trzeba wybrać :) 
- Po drugie, warto zrobić to w taki sposób, aby nie zajęło zbyt wiele czasu na zastanawianie się nad tym. Jeśli w trakcie konkursu uznasz, że któraś byłaby lepsza, to super. Oczywiście w ramach konkursu metryka zostanie jak wyżej, ale jak najbardziej przy okazji polecam zastanowić się, czy da się zrobić to lepiej.
- Po trzecie, [zasada Pareta](https://pl.wikipedia.org/wiki/Zasada_Pareta) naprawdę działa, czyli 20% wysiłku tworzy nam 80% wartości (i też można uznać, że działa to w drugą stronę). Nawet powiem więcej, warto to jeszcze podkręcić do kwadratu i koncentrować się na 4% wysiłku, aby uzyskać 64% wartości.

Zobacz na przykładzie. Sprawdziłem 4% najdroższych zwrotów i okazało się, że to 80% wartości do wszystkich zwrotów. Zobacz, jak rozkład wygląda na wykresie.
![](../images/cumsum.png)

Dlatego w tym przypadku warto bardzo karać przypadki większe, `mean_square_error` robi to całkiem dobrze.


### Kaggle
Konkurs będzie dostępny na Kaggle - linku do konkursu szukaj w mailu. 



### Regulamin
1. Konkurs rozpoczyna się **31 marca 2020 r.** i kończy się w niedzielę, **12 kwietnia 2020 r. o 23:59** (UTC).
2. Zbiór danych został podzielony na dwie części: publiczny i prywatny. Publiczny zbiór danych zawiera cechy + odpowiedź, prywatny zbiór tylko cechy. Należy zrobić predykcję na prywatnym zbiorze i wysłać na Kaggle. Gdy skończy się konkurs, to ostateczny wynik będzie na zbiorze prywatnym.
3. Dane są przygotowane w celach edukacyjnych, zakazane jest ich używanie w innym celu niż ten kurs oraz zakazane jest upublicznianie danych w dowolny sposób. Rozwiązanie ma być oparte tylko i wyłącznie przy użyciu uczenia maszynowego (trenowanie modelu na podstawie udostępnionych danych). Nie ma co kombinować, skup się na nauce :).
4. W tym konkursie jako absolwenci możecie łączyć się w drużyny. Każdy uczestnik / drużyna może dodać rozwiązanie (czyli zrobić tak zwany *submit*) do 5 prób rozwiązań w jeden dzień (dlatego warto zacząć wcześniej, będzie więcej prób).
5. Wszyscy absolwenci biorący udział w konkursie opublikują wszystkim swoje najlepsze rozwiązanie i udostępnią je podając link na Slacku.


### Dane

W danych jest **1 067 371** wierszy, które zostały podzielone prawie na równe części:
- `train_online_retail` (520 142 wierszy)
- `test_online_retail` (547 229 wierszy)

Uwaga! Zwróć uwagę, że w wierszach jest pojedynczy produkt, a zamówienie może się składać z kilku produktów. Docelowo będziemy prognozować kwotę do zwrotu dla danego zamówienia. **Twoim zadaniem jest zrobić predykcję wartości zamówienia do zwrotu dla zbioru testowego**. 


Jak robisz `submit` na Kaggle, to jest wyliczany wynik. Zwróć uwagę, że wynik jest wyliczany tylko na 30% danych testowych i ten wynik jest widoczny na tak zwanym "Public Leader Board". 

W praktyce to oznacza, że w Twoim zbiorze treningowym są odpowiedzi, ale masz tak trenować model, aby móc go zwalidować najpierw lokalnie (czyli użyć np. walidację krzyżową lub coś bardziej zaawansowanego). Następnie wrzucasz swój wynik na Kaggle (jako plik csv). W ciągu dnia masz maksymalnie 5 prób (czym wcześniej zaczniesz robić konkurs, tym więcej będziesz mieć prób). W trakcie konkursu Twój wyliczony wynik jest sprawdzony tylko na części danych (w tym przypadku 30%). To jest tak zwany Public Leader Board.

![](../images/public_private.png)

Gdy konkurs się skończy, wynik będzie przeliczony na wszystkich dostępnych danych testowych i będzie dostępny na tak zwanym "Private Leader Board". Dlaczego tak się robi? Chodzi o to, że Twoje rozwiązanie powinno być jak najbardziej stabilne na jak najmniejszej jak i większej próbce danych. Dlatego musisz uważnie robić walidację, bo pierwsze miejsce na Public Leader Board nie musi być pierwszym na Private Leader Board :) Uważaj na to!

![](../images/30_70.png)





In [ ]:
import pandas as pd
import numpy as np
np.random.seed(0)

from sklearn.metrics import mean_squared_error as mse
from sklearn.dummy import DummyRegressor

## Dane
To są prawdziwe dane.

**Uwaga!** Danych można używać tylko i wyłącznie w **celach edukacyjnych** (również nie można ich publikować lub dzielić się nimi z innymi)!

In [ ]:
train = pd.read_hdf('../input/train_online_retail.h5')
train.info()

W pliku `train_online_retail` jeden wiersz dotyczy jednego produktu. Jedno zamówienie może zawierać 1 lub więcej produktów.

Ostatecznie prognozowanie będzie per zamówienie, czyli należy to zrozumieć jako posortowanie wszystkich produktów po fakturze. Zróbmy to, aby lepiej było zrozumiałe, co należy zrobić.

In [ ]:
orders = train.groupby('invoice')['is_canceled', 'price_total'].sum()
orders.sample(15)

W wyniku tej operacji mamy sumaryczny koszt dla wszystkich produktów w danym zamówieniu. Zwróć uwagę na kolumnę `is_canceled`. Jak widzisz, ma ona teraz wartości albo 0 albo coś większego (np. 1., 2 itd). To nastąpiło na skutek tego, że w jednym zamówieniu możemy mieć więcej produktów. Każdy z nich został zwrócony, wynik to liczba, która w tym momencie tak naprawdę mówi o tym, ile było produktów w zamówieniu. Wybierz dowolny `invoice` powyżej, gdzie `is_canceled` jest większy niż zero. Na przykład dla `invoice=3046` mamy 2.0. 

In [ ]:
train[ train.invoice == 3046 ]

Jak widzisz mamy dwa produkty. Tutaj też ciekawostka Pythona, który automatycznie rzutuje `bool` do `int`. Zobacz, suma listy składającej się z dwóch `True` będzie 2 :).

In [ ]:
sum([True, True])

Zróbmy teraz małą kosmetyczną zmianę w naszym `orders`, aby `is_canceled` było albo `True` albo `False`.

In [ ]:
orders['is_canceled'] = orders['is_canceled'] > 0
orders.sample(15)

Twoim zadaniem jest sprognozować wartość zwrotów. Co to oznacza w praktyce? Jeśli zamówienie było anulowane, to wtedy trzeba  zwrócić to, co jest w kolumnie `price_total`, w innym przypadku jest zero (czyli niczego nie trzeba zwracać, bo klient zostawił produkt u siebie). W ten sposób powstaje nam kolumna `total_return`, to jest to, co będziesz przewidywać :). 

In [ ]:
orders['total_return'] = orders['price_total'] * orders['is_canceled'] 
orders.sample(20)

To samo zrobiłem i zapisałem do pliku `csv` i tak powstał `../input/orders_train.csv`.

In [ ]:
orders_train =  pd.read_csv('../input/orders_train.csv')
orders_train.sample(20)

Taki sam plik jest dla testu, czyli `../input/orders_test.csv`. O tym będzie poniżej.

Zróbmy teraz najprostszy model, aby jak najszybciej rozpocząć przygodę z Kaggle.

## Basic Model

In [ ]:
feats = ['invoice'] # w tym przypadku wszystko jedno jaka kolumna
X = orders_train[feats].values
y = orders_train['total_return'].values

model = DummyRegressor(strategy='mean')
model.fit(X, y)
y_pred = model.predict(X)

mse(y, y_pred)

Pierwszy wynik jest ok. **1 492 585**. Teraz przygotujmy dane dla `submit`'u!

## Submit

In [ ]:
orders_test =  pd.read_csv('../input/orders_test.csv')

X = orders_test[feats].values
y_pred = model.predict(X)

## Submission
Trzeba przygotować plik, który będzie zawierał: `id` i `readmitted`, następnie zaloguj się na Kaggle, wejdź na stronę konkursu i dodaj swój wynik :). Koniecznie napisz na Slacku, że już "pierwsze koty za płoty" :).

In [ ]:
orders_test['total_return'] = y_pred
orders_test[ ['invoice', 'total_return'] ].to_csv('../output/submit_dummy_model.csv', index=False) 

![](../images/submit.png)

Teraz już zastosuj swoją wiedzę, którą masz.
Spróbuj znaleźć lepsze cechy, lepszy model i lepsze parametry.

Powodzenia!

In [ ]:
#YOUR CODE HERE

## Dane:

- `invoice` - nr. faktury
- `stock_code` - produkt ID
- `description` - opis produktu
- `quantity` - liczba kupionych tych samych produktów w ramach jednej transakcji
- `invoice_date` - data wystawienie faktury
- `price_unit` - cena jednostkowa produktu za sztukę w funtach
- `price_total` - cena razem `price_unit * quantity` 
- `customer_id` - klient ID
- `country` - nazwa kraju, z którego klient kupował
- `is_canceled` - czy faktura była anulowana
- `is_test` - czy to zbiór testowy

In [ ]:
### YOUR CODE HERE